<a href="https://colab.research.google.com/github/your-repo/Speaker-diarization-/blob/main/Enhanced_Diarization_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎙️ Enhanced Multi-Channel Speaker Diarization Training

**Objectif :** Entraîner un système de diarization de locuteurs avancé sur le corpus AMI  
**Architecture :** TCN multi-échelle avec attention, classification de locuteurs et gestion mémoire optimisée

---

## 📋 Table des Matières
1. [Configuration de l'Environnement](#setup)
2. [Téléchargement du Corpus AMI](#data)
3. [Préparation des Données](#preprocessing)
4. [Modèle et Configuration](#model)
5. [Entraînement](#training)
6. [Évaluation](#evaluation)
7. [Sauvegarde et Visualisations](#results)

## 🔧 1. Configuration de l'Environnement

Installation de Conda et des dépendances nécessaires pour l'entraînement.

In [ ]:
# Installation de Conda sur Google Colab
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda --version

import sys
sys.path.append('/usr/local/lib/python3.10/site-packages')

In [ ]:
# Création de l'environnement conda pour la diarization
!conda create -n diarization python=3.9 -y
!conda activate diarization

# Installation des dépendances principales via conda
!conda install -n diarization pytorch torchaudio cudatoolkit=11.8 -c pytorch -c nvidia -y
!conda install -n diarization numpy scipy scikit-learn matplotlib seaborn pandas -y

# Activation de l'environnement dans le notebook
import os
os.environ['CONDA_DEFAULT_ENV'] = 'diarization'
os.environ['PATH'] = '/usr/local/envs/diarization/bin:' + os.environ['PATH']

In [ ]:
# Installation des dépendances spécialisées
!pip install wandb optuna tqdm psutil
!pip install librosa soundfile
!pip install speechbrain

# Vérification des installations
import torch
import torchaudio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

### 📂 Clonage du Répertoire et Configuration

In [ ]:
# Cloner le projet (remplacez par votre URL de repository)
!git clone https://github.com/your-username/Speaker-diarization-.git
%cd Speaker-diarization-

# Vérifier la structure du projet
!ls -la
!ls -la src/

In [ ]:
# Configuration des chemins et imports
import sys
sys.path.append('./src')

# Créer les dossiers nécessaires
!mkdir -p data/ami_corpus/audio
!mkdir -p data/ami_corpus/annotations
!mkdir -p models/checkpoints
!mkdir -p results/logs
!mkdir -p results/figures

# Variables globales
DATA_DIR = Path('./data/ami_corpus')
AUDIO_DIR = DATA_DIR / 'audio'
ANNOTATION_DIR = DATA_DIR / 'annotations'
MODEL_DIR = Path('./models/checkpoints')
RESULTS_DIR = Path('./results')

print(f"Répertoires configurés:")
print(f"- Audio: {AUDIO_DIR}")
print(f"- Annotations: {ANNOTATION_DIR}")
print(f"- Modèles: {MODEL_DIR}")
print(f"- Résultats: {RESULTS_DIR}")

## 📊 2. Téléchargement et Préparation du Corpus AMI

Le corpus AMI contient des enregistrements de réunions avec annotations temporelles des locuteurs.

In [ ]:
# Téléchargement du corpus AMI (version réduite pour Colab)
# URL officielle: https://groups.inf.ed.ac.uk/ami/corpus/

import urllib.request
import zipfile
from tqdm import tqdm

def download_with_progress(url, filename):
    """Télécharge un fichier avec barre de progression."""
    def progress_hook(blocknum, blocksize, totalsize):
        readsofar = blocknum * blocksize
        if totalsize > 0:
            percent = readsofar * 1e2 / totalsize
            s = f"\r{percent:5.1f}% {readsofar:,} / {totalsize:,} bytes"
            sys.stderr.write(s)
            if readsofar >= totalsize:
                sys.stderr.write("\n")
        else:
            sys.stderr.write(f"\rread {readsofar:,}")
    
    urllib.request.urlretrieve(url, filename, progress_hook)

# URLs pour le corpus AMI (échantillon)
ami_urls = {
    'audio_sample': 'https://groups.inf.ed.ac.uk/ami/AMICorpusAnnotations/ami_public_manual_1.6.2.zip',
    # Ajoutez d'autres URLs selon vos besoins
}

print("📥 Téléchargement des annotations AMI...")
annotation_file = DATA_DIR / 'ami_annotations.zip'
try:
    download_with_progress(ami_urls['audio_sample'], str(annotation_file))
    print("✅ Téléchargement terminé!")
    
    # Extraction
    with zipfile.ZipFile(annotation_file, 'r') as zip_ref:
        zip_ref.extractall(ANNOTATION_DIR)
    print("✅ Extraction terminée!")
    
except Exception as e:
    print(f"⚠️ Erreur de téléchargement: {e}")
    print("Utilisation des données d'exemple...")

In [ ]:
# Préparation alternative: utilisation des données existantes du projet
# Si le téléchargement AMI échoue, on utilise les données de démonstration

def setup_demo_data():
    """Configure des données de démonstration pour l'entraînement."""
    print("🔧 Configuration des données de démonstration...")
    
    # Vérifier si les données AMI préparées existent
    if Path('./annot_prepare/results/rttm').exists():
        print("✅ Données AMI préparées trouvées!")
        return './annot_prepare/results/meta', './annot_prepare/results/rttm'
    
    # Sinon, créer des données synthétiques pour la démonstration
    print("🎭 Création de données synthétiques...")
    
    # Créer des fichiers audio fictifs (spectrogrammes aléatoires)
    import torch
    import json
    
    # Paramètres audio
    sample_rate = 16000
    duration = 60  # 60 secondes par fichier
    n_channels = 8  # 8 microphones
    n_files = 20   # 20 fichiers pour la démo
    
    audio_files = []
    rttm_files = []
    
    for i in range(n_files):
        file_id = f"demo_meeting_{i:03d}"
        
        # Créer audio synthétique
        audio_data = torch.randn(n_channels, sample_rate * duration)
        audio_path = AUDIO_DIR / f"{file_id}.pt"
        torch.save(audio_data, audio_path)
        audio_files.append(str(audio_path))
        
        # Créer annotations RTTM synthétiques
        rttm_content = []
        n_speakers = np.random.randint(2, 5)  # 2-4 locuteurs
        
        for spk_idx in range(n_speakers):
            speaker_id = f"speaker_{spk_idx}"
            
            # Générer segments aléatoires
            n_segments = np.random.randint(3, 8)
            for seg_idx in range(n_segments):
                start_time = np.random.uniform(0, duration - 10)
                segment_duration = np.random.uniform(2, 8)
                end_time = min(start_time + segment_duration, duration)
                
                # Format RTTM: SPEAKER file_id 1 start_time duration <NA> <NA> speaker_id <NA> <NA>
                rttm_line = f"SPEAKER {file_id} 1 {start_time:.3f} {segment_duration:.3f} <NA> <NA> {speaker_id} <NA> <NA>"
                rttm_content.append(rttm_line)
        
        # Sauvegarder RTTM
        rttm_path = ANNOTATION_DIR / f"{file_id}.rttm"
        with open(rttm_path, 'w') as f:
            f.write('\n'.join(rttm_content))
        rttm_files.append(str(rttm_path))
    
    print(f"✅ Créé {n_files} fichiers de démonstration")
    print(f"   - Audio: {len(audio_files)} fichiers")
    print(f"   - RTTM: {len(rttm_files)} fichiers")
    
    return str(AUDIO_DIR), str(ANNOTATION_DIR)

# Configurer les données
audio_dir, rttm_dir = setup_demo_data()

print(f"\n📂 Répertoires configurés:")
print(f"   Audio: {audio_dir}")
print(f"   RTTM: {rttm_dir}")

### 📊 Division des Données (Train/Eval)

Division stratifiée du corpus AMI selon les bonnes pratiques.

In [ ]:
def create_ami_splits(audio_dir, rttm_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """
    Divise le corpus AMI en ensembles d'entraînement, validation et test.
    
    Args:
        audio_dir: Répertoire des fichiers audio
        rttm_dir: Répertoire des annotations RTTM
        train_ratio: Proportion pour l'entraînement
        val_ratio: Proportion pour la validation
        test_ratio: Proportion pour le test
    
    Returns:
        dict: Dictionnaire avec les listes de fichiers pour chaque split
    """
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Les ratios doivent sommer à 1.0"
    
    # Lister tous les fichiers disponibles
    audio_path = Path(audio_dir)
    rttm_path = Path(rttm_dir)
    
    # Trouver les fichiers audio
    audio_extensions = ['.wav', '.pt', '.flac', '.mp3']
    audio_files = []
    for ext in audio_extensions:
        audio_files.extend(list(audio_path.glob(f'*{ext}')))
    
    # Vérifier la correspondance audio-RTTM
    valid_pairs = []
    for audio_file in audio_files:
        base_name = audio_file.stem
        rttm_file = rttm_path / f"{base_name}.rttm"
        
        if rttm_file.exists():
            valid_pairs.append({
                'base_name': base_name,
                'audio_file': str(audio_file),
                'rttm_file': str(rttm_file)
            })
    
    print(f"📊 Trouvé {len(valid_pairs)} paires audio-RTTM valides")
    
    if len(valid_pairs) == 0:
        raise ValueError("Aucune paire audio-RTTM valide trouvée!")
    
    # Mélanger et diviser
    import random
    random.seed(42)  # Pour la reproductibilité
    random.shuffle(valid_pairs)
    
    n_total = len(valid_pairs)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)
    
    train_files = valid_pairs[:n_train]
    val_files = valid_pairs[n_train:n_train + n_val]
    test_files = valid_pairs[n_train + n_val:]
    
    splits = {
        'train': train_files,
        'validation': val_files,
        'test': test_files
    }
    
    # Statistiques
    print(f"\n📈 Division des données:")
    for split_name, files in splits.items():
        print(f"   {split_name:>10}: {len(files):>3} fichiers ({len(files)/n_total*100:.1f}%)")
    
    # Sauvegarder les splits
    splits_file = DATA_DIR / 'data_splits.json'
    with open(splits_file, 'w') as f:
        json.dump(splits, f, indent=2)
    
    print(f"\n💾 Splits sauvegardés dans: {splits_file}")
    return splits

# Créer les splits
data_splits = create_ami_splits(audio_dir, rttm_dir, 
                               train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

# Vérifier le contenu
print("\n🔍 Exemple de fichiers par split:")
for split_name, files in data_splits.items():
    if files:
        print(f"\n{split_name.upper()}:")
        for i, file_info in enumerate(files[:3]):  # Afficher les 3 premiers
            print(f"  {i+1}. {file_info['base_name']}")
        if len(files) > 3:
            print(f"  ... et {len(files)-3} autres")

## 🛠️ 3. Préparation des Données et Extraction de Caractéristiques

Extraction des caractéristiques multi-canaux: LPS, IPD, AF

In [ ]:
# Import des modules du projet
try:
    from tcn_diarization_model import DiarizationTCN
    from optimized_dataset import DiarizationDataset, AudioFeatureExtractor
    from optimized_dataloader import create_optimized_dataloaders, MemoryAwareDataLoader
    from diarization_losses import MultiTaskDiarizationLoss, create_loss_function
    from metrics import DiarizationMetrics
    from improved_trainer import ImprovedDiarizationTrainer, MemoryMonitor
    
    print("✅ Tous les modules importés avec succès!")
    
except ImportError as e:
    print(f"❌ Erreur d'import: {e}")
    print("📥 Téléchargement des fichiers depuis le repository...")
    
    # Si les imports échouent, créer des versions simplifiées pour la démo
    !wget -q https://raw.githubusercontent.com/your-repo/Speaker-diarization-/main/src/tcn_diarization_model.py
    !wget -q https://raw.githubusercontent.com/your-repo/Speaker-diarization-/main/src/optimized_dataset.py
    !wget -q https://raw.githubusercontent.com/your-repo/Speaker-diarization-/main/src/optimized_dataloader.py
    !wget -q https://raw.githubusercontent.com/your-repo/Speaker-diarization-/main/src/diarization_losses.py
    !wget -q https://raw.githubusercontent.com/your-repo/Speaker-diarization-/main/src/metrics.py
    !wget -q https://raw.githubusercontent.com/your-repo/Speaker-diarization-/main/src/improved_trainer.py
    
    # Réessayer l'import
    from tcn_diarization_model import DiarizationTCN
    from optimized_dataset import DiarizationDataset, AudioFeatureExtractor
    from optimized_dataloader import create_optimized_dataloaders, MemoryAwareDataLoader
    from diarization_losses import MultiTaskDiarizationLoss, create_loss_function
    from metrics import DiarizationMetrics
    from improved_trainer import ImprovedDiarizationTrainer, MemoryMonitor
    
    print("✅ Modules téléchargés et importés!")

In [ ]:
# Test de l'extraction de caractéristiques
print("🧪 Test de l'extraction de caractéristiques...")

# Créer un extracteur de caractéristiques
feature_extractor = AudioFeatureExtractor(
    sample_rate=16000,
    n_fft=512,
    hop_length=256
)

# Créer des données audio fictives (8 canaux)
n_channels = 8
duration = 4.0  # 4 secondes
sample_rate = 16000
n_samples = int(duration * sample_rate)

# Simuler audio multi-canal avec du bruit et des signaux
waveforms = []
for ch in range(n_channels):
    # Signal de base + bruit
    base_signal = np.sin(2 * np.pi * 440 * np.linspace(0, duration, n_samples))  # 440 Hz
    noise = np.random.normal(0, 0.1, n_samples)
    # Ajouter un léger décalage temporel pour simuler la spatialisation
    delay_samples = int(0.001 * ch * sample_rate)  # 1ms de délai par canal
    delayed_signal = np.zeros(n_samples)
    if delay_samples < n_samples:
        delayed_signal[delay_samples:] = base_signal[:n_samples-delay_samples]
    
    final_signal = delayed_signal + noise
    waveforms.append(torch.tensor(final_signal, dtype=torch.float32))

print(f"📊 Audio généré: {n_channels} canaux, {duration}s, {sample_rate} Hz")

# Test d'extraction
features = feature_extractor.extract_features(waveforms)
print(f"✅ Caractéristiques extraites: {features.shape}")
print(f"   - Dimensions attendues: [771, ~{int(duration * sample_rate / 256)}]")
print(f"   - Dimensions obtenues: {list(features.shape)}")

# Analyser les caractéristiques
print(f"\n📈 Statistiques des caractéristiques:")
print(f"   - Min: {features.min():.3f}")
print(f"   - Max: {features.max():.3f}")
print(f"   - Moyenne: {features.mean():.3f}")
print(f"   - Std: {features.std():.3f}")

# Visualiser les caractéristiques
plt.figure(figsize=(15, 8))

# Spectrogramme des caractéristiques
plt.subplot(2, 2, 1)
plt.imshow(features.numpy()[:100, :], aspect='auto', origin='lower')
plt.title('Caractéristiques LPS (première partie)')
plt.xlabel('Temps (frames)')
plt.ylabel('Fréquence')
plt.colorbar()

plt.subplot(2, 2, 2)
plt.imshow(features.numpy()[257:357, :], aspect='auto', origin='lower')
plt.title('Caractéristiques IPD (première partie)')
plt.xlabel('Temps (frames)')
plt.ylabel('Fréquence')
plt.colorbar()

plt.subplot(2, 2, 3)
plt.imshow(features.numpy()[500:600, :], aspect='auto', origin='lower')
plt.title('Caractéristiques AF (première partie)')
plt.xlabel('Temps (frames)')
plt.ylabel('Fréquence')
plt.colorbar()

plt.subplot(2, 2, 4)
plt.plot(features.numpy().mean(axis=0))
plt.title('Énergie moyenne par frame')
plt.xlabel('Temps (frames)')
plt.ylabel('Énergie moyenne')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'feature_extraction_test.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Test d'extraction de caractéristiques terminé!")

## 🧠 4. Configuration du Modèle et Entraînement

Configuration optimale pour le corpus AMI avec les meilleures pratiques.

In [ ]:
# Configuration optimisée pour AMI corpus
config = {
    # === MODÈLE ===
    'model': {
        'input_dim': 771,  # LPS (257) + IPD (257*4) + AF (257*4) = 2313 → 771 après agrégation
        'hidden_channels': [256, 256, 256, 512, 512],  # Architecture TCN multi-échelle
        'kernel_size': 3,
        'num_speakers': 4,  # AMI corpus a typiquement 3-4 locuteurs
        'dropout': 0.2,
        'use_attention': True,  # Auto-attention pour dépendances long-terme
        'use_speaker_classifier': True,  # Classification de locuteurs
        'embedding_dim': 256
    },
    
    # === FONCTION DE PERTE ===
    'loss': {
        'type': 'multitask',
        'vad_weight': 1.0,
        'osd_weight': 1.0,
        'consistency_weight': 0.1,
        'use_pit': True,  # Permutation Invariant Training
        'use_focal': True,  # Focal Loss pour données déséquilibrées
        'focal_gamma': 2.0,
        'num_speakers': 4
    },
    
    # === OPTIMISEUR ===
    'optimizer': {
        'type': 'adamw',
        'lr': 1e-3,  # Learning rate initial
        'weight_decay': 1e-4,
        'betas': (0.9, 0.999)
    },
    
    # === PLANIFICATEUR LR ===
    'scheduler': {
        'type': 'onecycle',  # OneCycleLR pour convergence rapide
        'steps_per_epoch': 100,  # Sera mis à jour automatiquement
        'pct_start': 0.3  # 30% montée, 70% descente
    },
    
    # === ENTRAÎNEMENT ===
    'training': {
        'epochs': 50,  # Réduit pour Colab
        'batch_size': 8,  # Adapté à la mémoire Colab
        'num_workers': 2  # Moins de workers pour éviter les problèmes mémoire
    },
    
    # === DONNÉES ===
    'data': {
        'segment_duration': 4.0,  # Segments de 4 secondes
        'sample_rate': 16000,
        'train_split': 0.7,
        'max_segments': 1000  # Limite pour Colab
    },
    
    # === OPTIMISATIONS AVANCÉES ===
    'accumulation_steps': 4,  # Batch effectif = 8*4 = 32
    'use_amp': True,  # Précision mixte
    'grad_clip_norm': 1.0,
    'patience': 10,
    'save_every': 5,
    
    # === MONITORING ===
    'use_wandb': True,  # Weights & Biases (optionnel)
    'project_name': 'ami-speaker-diarization',
    'memory_threshold': 0.85,  # Gestion mémoire Colab
    'adaptive_batch': True,
    'speaker_loss_weight': 0.5,
    
    # === CHEMINS ===
    'save_dir': str(MODEL_DIR),
    'results_dir': str(RESULTS_DIR)
}

print("⚙️ Configuration créée avec les paramètres suivants:")
print(f"   - Architecture: TCN {config['model']['hidden_channels']}")
print(f"   - Batch size: {config['training']['batch_size']} (effectif: {config['training']['batch_size'] * config['accumulation_steps']})")
print(f"   - Epochs: {config['training']['epochs']}")
print(f"   - Learning rate: {config['optimizer']['lr']}")
print(f"   - Précision mixte: {config['use_amp']}")
print(f"   - Classification locuteurs: {config['model']['use_speaker_classifier']}")

# Sauvegarder la configuration
config_file = MODEL_DIR / 'training_config.json'
with open(config_file, 'w') as f:
    json.dump(config, f, indent=2)

print(f"\n💾 Configuration sauvegardée: {config_file}")

In [ ]:
# Configuration de Weights & Biases (optionnel)
import wandb

use_wandb = config.get('use_wandb', False)

if use_wandb:
    try:
        # Connexion à wandb (nécessite un compte gratuit)
        wandb.login()
        
        # Initialisation du projet
        wandb.init(
            project=config['project_name'],
            config=config,
            name=f"ami-tcn-{torch.cuda.get_device_name(0).replace(' ', '-') if torch.cuda.is_available() else 'cpu'}",
            tags=['ami-corpus', 'tcn', 'multi-channel', 'colab'],
            notes="Entraînement sur corpus AMI avec architecture TCN améliorée"
        )
        
        print("✅ Weights & Biases configuré!")
        print(f"📊 Dashboard: {wandb.run.url}")
        
    except Exception as e:
        print(f"⚠️ Erreur wandb: {e}")
        print("📈 Entraînement sans monitoring wandb...")
        config['use_wandb'] = False
else:
    print("📈 Entraînement sans monitoring wandb (désactivé dans config)")

## 🚀 5. Entraînement du Modèle

Entraînement avec toutes les optimisations: gestion mémoire, precision mixte, accumulation de gradients.

In [ ]:
# Création des DataLoaders optimisés
print("🔄 Création des DataLoaders...")

try:
    # Créer les DataLoaders avec gestion mémoire
    train_loader, val_loader = create_optimized_dataloaders(
        audio_dir=audio_dir,
        rttm_dir=rttm_dir,
        batch_size=config['training']['batch_size'],
        train_split=config['data']['train_split'],
        num_workers=config['training']['num_workers'],
        segment_duration=config['data']['segment_duration'],
        sample_rate=config['data']['sample_rate'],
        max_segments=config['data']['max_segments'],
        memory_threshold=config['memory_threshold'],
        adaptive_batch=config['adaptive_batch'],
        accumulation_steps=config['accumulation_steps']
    )
    
    print(f"✅ DataLoaders créés avec succès!")
    print(f"   - Train batches: {len(train_loader)}")
    print(f"   - Validation batches: {len(val_loader)}")
    
    # Mettre à jour la configuration avec le nombre réel de steps
    config['scheduler']['steps_per_epoch'] = len(train_loader)
    
    # Test d'un batch
    print("\n🧪 Test d'un batch d'entraînement...")
    for batch_idx, batch in enumerate(train_loader):
        print(f"   Batch {batch_idx}:")
        print(f"     - Features: {batch['features'].shape}")
        print(f"     - VAD labels: {batch['vad_labels'].shape}")
        print(f"     - OSD labels: {batch['osd_labels'].shape}")
        
        # Vérifier les dimensions
        assert batch['features'].shape[1] == 771, f"Dimension features incorrecte: {batch['features'].shape[1]} != 771"
        assert batch['vad_labels'].shape[-1] == 4, f"Nombre de locuteurs incorrect: {batch['vad_labels'].shape[-1]} != 4"
        
        print(f"     ✅ Dimensions correctes!")
        break
        
except Exception as e:
    print(f"❌ Erreur création DataLoaders: {e}")
    
    # Fallback: créer un dataset simple pour la démonstration
    print("🔄 Création d'un dataset de démonstration...")
    
    class DemoDataset(torch.utils.data.Dataset):
        def __init__(self, size=100, input_dim=771, seq_len=250, num_speakers=4):
            self.size = size
            self.input_dim = input_dim
            self.seq_len = seq_len
            self.num_speakers = num_speakers
            
        def __len__(self):
            return self.size
            
        def __getitem__(self, idx):
            # Créer des données synthétiques réalistes
            features = torch.randn(self.input_dim, self.seq_len) * 0.5
            
            # VAD labels avec activité réaliste
            vad_labels = torch.zeros(self.seq_len, self.num_speakers)
            for spk in range(self.num_speakers):
                if np.random.random() > 0.3:  # 70% chance d'activité
                    start = np.random.randint(0, self.seq_len // 2)
                    end = np.random.randint(start + 10, self.seq_len)
                    vad_labels[start:end, spk] = 1.0
            
            # OSD labels basés sur chevauchement
            osd_labels = (vad_labels.sum(dim=1) > 1).float()
            
            return {
                'features': features,
                'vad_labels': vad_labels,
                'osd_labels': osd_labels,
                'segment_id': idx
            }
    
    # Créer datasets de démo
    train_dataset = DemoDataset(size=200)
    val_dataset = DemoDataset(size=50)
    
    # DataLoaders de démo
    train_loader = torch.utils.data.DataLoader(
        train_dataset, 
        batch_size=config['training']['batch_size'],
        shuffle=True,
        num_workers=0  # Pas de multiprocessing pour éviter les erreurs
    )
    
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=config['training']['batch_size'],
        shuffle=False,
        num_workers=0
    )
    
    print(f"✅ Dataset de démo créé:")
    print(f"   - Train: {len(train_loader)} batches")
    print(f"   - Val: {len(val_loader)} batches")
    
    config['scheduler']['steps_per_epoch'] = len(train_loader)

In [ ]:
# Initialisation du trainer avancé
print("🧠 Initialisation du trainer...")

try:
    # Créer le trainer avec toutes les améliorations
    trainer = ImprovedDiarizationTrainer(config)
    
    print(f"✅ Trainer initialisé avec succès!")
    print(f"   - Modèle: {trainer.model.get_num_params():,} paramètres")
    print(f"   - Device: {trainer.device}")
    print(f"   - Précision mixte: {trainer.use_amp}")
    print(f"   - Accumulation gradients: {trainer.accumulation_steps}")
    
    # Test du forward pass
    print("\n🧪 Test du modèle...")
    model = trainer.model
    model.eval()
    
    with torch.no_grad():
        # Test avec un batch de données
        test_input = torch.randn(2, 771, 250).to(trainer.device)
        
        # Forward pass simple
        vad_out, osd_out = model(test_input)
        print(f"   Forward simple: VAD {vad_out.shape}, OSD {osd_out.shape}")
        
        # Forward avec embeddings
        vad_out, osd_out, embeddings, speaker_logits = model(test_input, return_embeddings=True)
        print(f"   Forward complet: VAD {vad_out.shape}, OSD {osd_out.shape}")
        print(f"                   Embeddings {embeddings.shape}, Speaker {speaker_logits.shape}")
    
    print("✅ Modèle fonctionne correctement!")
    
except Exception as e:
    print(f"❌ Erreur initialisation trainer: {e}")
    
    # Fallback: créer un trainer simple
    print("🔄 Création d'un trainer simple...")
    
    # Créer juste le modèle
    model = DiarizationTCN(
        input_dim=config['model']['input_dim'],
        hidden_channels=config['model']['hidden_channels'],
        num_speakers=config['model']['num_speakers'],
        use_speaker_classifier=config['model']['use_speaker_classifier']
    )
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Optimiseur simple
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config['optimizer']['lr'],
        weight_decay=config['optimizer']['weight_decay']
    )
    
    # Critère de perte
    criterion = MultiTaskDiarizationLoss(
        use_pit=config['loss']['use_pit'],
        use_focal=config['loss']['use_focal'],
        num_speakers=config['model']['num_speakers']
    )
    
    print(f"✅ Trainer simple créé:")
    print(f"   - Paramètres: {sum(p.numel() for p in model.parameters()):,}")
    print(f"   - Device: {device}")
    
    # Pour compatibilité avec le code d'entraînement
    class SimpleTrainer:
        def __init__(self, model, optimizer, criterion, device):
            self.model = model
            self.optimizer = optimizer
            self.criterion = criterion
            self.device = device
            self.use_amp = config.get('use_amp', False)
            
        def train_epoch(self, train_loader):
            # Implémentation simple d'entraînement
            self.model.train()
            total_loss = 0
            
            for batch_idx, batch in enumerate(train_loader):
                features = batch['features'].to(self.device)
                vad_labels = batch['vad_labels'].to(self.device)
                osd_labels = batch['osd_labels'].to(self.device)
                
                self.optimizer.zero_grad()
                
                vad_pred, osd_pred = self.model(features)
                loss_dict = self.criterion(vad_pred, osd_pred, vad_labels, osd_labels)
                loss = loss_dict['total_loss']
                
                loss.backward()
                self.optimizer.step()
                
                total_loss += loss.item()
                
                if batch_idx % 20 == 0:
                    print(f"   Batch {batch_idx}/{len(train_loader)}: Loss {loss.item():.4f}")
            
            return {'loss': total_loss / len(train_loader)}
    
    trainer = SimpleTrainer(model, optimizer, criterion, device)

In [ ]:
# Démarrage de l'entraînement
print("🚀 DÉMARRAGE DE L'ENTRAÎNEMENT")
print("=" * 50)

# Monitoring mémoire
memory_monitor = MemoryMonitor()
initial_memory = memory_monitor.get_memory_info()

print(f"💾 Mémoire initiale:")
print(f"   - RAM: {initial_memory['ram_percent']:.1f}%")
print(f"   - GPU: {initial_memory['gpu_percent']:.1f}%")

# Configuration d'entraînement
num_epochs = config['training']['epochs']
save_every = config.get('save_every', 5)

# Historique des métriques
train_losses = []
val_losses = []
best_val_loss = float('inf')

print(f"\n📋 Configuration d'entraînement:")
print(f"   - Epochs: {num_epochs}")
print(f"   - Batch size: {config['training']['batch_size']}")
print(f"   - Learning rate: {config['optimizer']['lr']}")
print(f"   - Sauvegarde chaque {save_every} epochs")

print("\n" + "="*50)
print("DÉBUT DE L'ENTRAÎNEMENT")
print("="*50)

import time
from datetime import datetime

# Boucle d'entraînement principale
start_time = time.time()

try:
    for epoch in range(num_epochs):
        epoch_start = time.time()
        
        print(f"\n🔄 Epoch {epoch+1}/{num_epochs} - {datetime.now().strftime('%H:%M:%S')}")
        print("-" * 40)
        
        # Phase d'entraînement
        if hasattr(trainer, 'train_epoch'):
            # Utiliser le trainer avancé
            train_metrics = trainer.train_epoch(train_loader)
            train_loss = train_metrics['total_loss']
        else:
            # Entraînement simple
            trainer.model.train()
            total_loss = 0
            num_batches = 0
            
            for batch_idx, batch in enumerate(train_loader):
                features = batch['features'].to(trainer.device)
                vad_labels = batch['vad_labels'].to(trainer.device)
                osd_labels = batch['osd_labels'].to(trainer.device)
                
                trainer.optimizer.zero_grad()
                
                vad_pred, osd_pred = trainer.model(features)
                loss_dict = trainer.criterion(vad_pred, osd_pred, vad_labels, osd_labels)
                loss = loss_dict['total_loss']
                
                loss.backward()
                trainer.optimizer.step()
                
                total_loss += loss.item()
                num_batches += 1
                
                if batch_idx % 20 == 0:
                    print(f"   Batch {batch_idx}/{len(train_loader)}: Loss {loss.item():.4f}")
            
            train_loss = total_loss / num_batches
        
        train_losses.append(train_loss)
        
        # Phase de validation
        print(f"\n📊 Validation...")
        trainer.model.eval()
        val_loss = 0
        num_val_batches = 0
        
        with torch.no_grad():
            for batch in val_loader:
                features = batch['features'].to(trainer.device)
                vad_labels = batch['vad_labels'].to(trainer.device)
                osd_labels = batch['osd_labels'].to(trainer.device)
                
                vad_pred, osd_pred = trainer.model(features)
                loss_dict = trainer.criterion(vad_pred, osd_pred, vad_labels, osd_labels)
                val_loss += loss_dict['total_loss'].item()
                num_val_batches += 1
        
        val_loss = val_loss / num_val_batches if num_val_batches > 0 else 0
        val_losses.append(val_loss)
        
        # Monitoring mémoire
        current_memory = memory_monitor.get_memory_info()
        
        # Résumé de l'epoch
        epoch_time = time.time() - epoch_start
        print(f"\n📈 Epoch {epoch+1} Résultats:")
        print(f"   - Train Loss: {train_loss:.4f}")
        print(f"   - Val Loss: {val_loss:.4f}")
        print(f"   - Temps: {epoch_time:.1f}s")
        print(f"   - Mémoire GPU: {current_memory['gpu_percent']:.1f}%")
        
        # Sauvegarde du meilleur modèle
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_path = MODEL_DIR / 'best_model.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': trainer.model.state_dict(),
                'optimizer_state_dict': trainer.optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'config': config
            }, best_model_path)
            print(f"   ✅ Nouveau meilleur modèle sauvé! (Loss: {val_loss:.4f})")
        
        # Sauvegarde périodique
        if (epoch + 1) % save_every == 0:
            checkpoint_path = MODEL_DIR / f'checkpoint_epoch_{epoch+1}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': trainer.model.state_dict(),
                'optimizer_state_dict': trainer.optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'train_losses': train_losses,
                'val_losses': val_losses,
                'config': config
            }, checkpoint_path)
            print(f"   💾 Checkpoint sauvé: {checkpoint_path.name}")
        
        # Logging wandb
        if config.get('use_wandb', False) and 'wandb' in locals():
            wandb.log({
                'epoch': epoch,
                'train_loss': train_loss,
                'val_loss': val_loss,
                'best_val_loss': best_val_loss,
                'gpu_memory_percent': current_memory['gpu_percent'],
                'epoch_time': epoch_time
            })
        
        # Nettoyage mémoire
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

except KeyboardInterrupt:
    print("\n⏹️ Entraînement interrompu par l'utilisateur")
except Exception as e:
    print(f"\n❌ Erreur durant l'entraînement: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    total_time = time.time() - start_time
    print(f"\n⏱️ Temps total d'entraînement: {total_time/60:.1f} minutes")
    print(f"🎯 Meilleure validation loss: {best_val_loss:.4f}")
    
    # Fermeture wandb
    if config.get('use_wandb', False) and 'wandb' in locals():
        wandb.finish()
    
print("\n" + "="*50)
print("ENTRAÎNEMENT TERMINÉ")
print("="*50)

## 📊 6. Évaluation et Métriques

Évaluation complète avec métriques de diarization standard.

In [ ]:
# Évaluation complète du modèle
print("📊 ÉVALUATION DU MODÈLE")
print("=" * 40)

# Charger le meilleur modèle
best_model_path = MODEL_DIR / 'best_model.pth'

if best_model_path.exists():
    print(f"📂 Chargement du meilleur modèle: {best_model_path}")
    checkpoint = torch.load(best_model_path, map_location=trainer.device)
    trainer.model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"   - Epoch: {checkpoint['epoch']}")
    print(f"   - Val Loss: {checkpoint['val_loss']:.4f}")
else:
    print("⚠️ Pas de modèle sauvé, utilisation du modèle actuel")

# Initialiser les métriques
metrics_computer = DiarizationMetrics(num_speakers=config['model']['num_speakers'])

# Évaluation sur l'ensemble de validation
print("\n🧪 Évaluation sur l'ensemble de validation...")
trainer.model.eval()

all_vad_preds = []
all_vad_targets = []
all_osd_preds = []
all_osd_targets = []

eval_loss = 0
num_eval_batches = 0

with torch.no_grad():
    for batch_idx, batch in enumerate(val_loader):
        features = batch['features'].to(trainer.device)
        vad_labels = batch['vad_labels'].to(trainer.device)
        osd_labels = batch['osd_labels'].to(trainer.device)
        
        # Prédictions
        if hasattr(trainer.model, 'use_speaker_classifier') and trainer.model.use_speaker_classifier:
            vad_pred, osd_pred, embeddings, speaker_logits = trainer.model(features, return_embeddings=True)
        else:
            vad_pred, osd_pred = trainer.model(features)
        
        # Loss
        loss_dict = trainer.criterion(vad_pred, osd_pred, vad_labels, osd_labels)
        eval_loss += loss_dict['total_loss'].item()
        num_eval_batches += 1
        
        # Collecter pour métriques
        all_vad_preds.append(vad_pred.cpu())
        all_vad_targets.append(vad_labels.cpu())
        all_osd_preds.append(osd_pred.cpu())
        all_osd_targets.append(osd_labels.cpu())
        
        if batch_idx % 10 == 0:
            print(f"   Batch {batch_idx}/{len(val_loader)} évalué")

# Calculer métriques détaillées
print("\n📈 Calcul des métriques détaillées...")

vad_preds = torch.cat(all_vad_preds, dim=0)
vad_targets = torch.cat(all_vad_targets, dim=0)
osd_preds = torch.cat(all_osd_preds, dim=0)
osd_targets = torch.cat(all_osd_targets, dim=0)

print(f"   - Données évaluées: {vad_preds.shape[0]} échantillons")
print(f"   - Durée totale: {vad_preds.shape[0] * vad_preds.shape[1] * 0.02:.1f} secondes")

# Métriques principales
metrics = metrics_computer.compute_metrics(vad_preds, osd_preds, vad_targets, osd_targets)

print("\n🎯 RÉSULTATS D'ÉVALUATION")
print("=" * 30)
print(f"📊 Loss finale: {eval_loss / num_eval_batches:.4f}")
print(f"📊 DER (Diarization Error Rate): {metrics.get('der', 0):.2f}%")
print(f"📊 F1 Score global: {metrics.get('f1_score', 0):.3f}")
print(f"📊 Précision frame: {metrics.get('frame_precision', 0):.3f}")
print(f"📊 Rappel frame: {metrics.get('frame_recall', 0):.3f}")
print(f"📊 Jaccard Index: {metrics.get('jaccard_index', 0):.3f}")

# Métriques OSD
if 'osd_precision' in metrics:
    print(f"\n🔀 Détection de Chevauchement (OSD):")
    print(f"   - Précision OSD: {metrics['osd_precision']:.3f}")
    print(f"   - Rappel OSD: {metrics['osd_recall']:.3f}")
    print(f"   - F1 OSD: {metrics['osd_f1']:.3f}")

# Métriques par locuteur
print(f"\n👥 Métriques par Locuteur:")
for spk in range(config['model']['num_speakers']):
    if f'speaker_{spk}_f1' in metrics:
        print(f"   Locuteur {spk}: F1={metrics[f'speaker_{spk}_f1']:.3f}, "
              f"P={metrics[f'speaker_{spk}_precision']:.3f}, "
              f"R={metrics[f'speaker_{spk}_recall']:.3f}")

# Sauvegarder les métriques
metrics_file = RESULTS_DIR / 'evaluation_metrics.json'
with open(metrics_file, 'w') as f:
    # Convertir les tenseurs en listes pour JSON
    json_metrics = {k: (v.item() if torch.is_tensor(v) else v) for k, v in metrics.items()}
    json_metrics['eval_loss'] = eval_loss / num_eval_batches
    json.dump(json_metrics, f, indent=2)

print(f"\n💾 Métriques sauvées: {metrics_file}")

## 📈 7. Visualisations et Analyses

Génération de graphiques et visualisations des résultats.

In [ ]:
# Visualisations des résultats
print("📊 Génération des visualisations...")

# Configuration matplotlib
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

# === 1. Courbes d'entraînement ===
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Résultats d\'Entraînement - Speaker Diarization TCN', fontsize=16, fontweight='bold')

# Courbe de perte
axes[0, 0].plot(train_losses, label='Train Loss', color='blue', linewidth=2)
axes[0, 0].plot(val_losses, label='Validation Loss', color='red', linewidth=2)
axes[0, 0].set_title('Évolution de la Perte')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Zoom sur les dernières epochs
if len(train_losses) > 10:
    start_idx = max(0, len(train_losses) - 20)
    axes[0, 1].plot(range(start_idx, len(train_losses)), train_losses[start_idx:], 
                   label='Train Loss', color='blue', linewidth=2)
    axes[0, 1].plot(range(start_idx, len(val_losses)), val_losses[start_idx:], 
                   label='Validation Loss', color='red', linewidth=2)
    axes[0, 1].set_title('Convergence (Dernières Epochs)')
else:
    axes[0, 1].plot(train_losses, label='Train Loss', color='blue', linewidth=2)
    axes[0, 1].plot(val_losses, label='Validation Loss', color='red', linewidth=2)
    axes[0, 1].set_title('Évolution de la Perte (Toutes Epochs)')

axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# === 2. Analyse des prédictions ===
# Prendre un échantillon pour visualisation
sample_idx = 0
sample_vad_pred = vad_preds[sample_idx].numpy()  # [time, speakers]
sample_vad_target = vad_targets[sample_idx].numpy()
sample_osd_pred = osd_preds[sample_idx].numpy()  # [time]
sample_osd_target = osd_targets[sample_idx].numpy()

# Activité des locuteurs (prédictions vs vérité terrain)
time_frames = np.arange(len(sample_vad_pred)) * 0.02  # Conversion en secondes

# Subplot pour VAD
axes[1, 0].imshow(sample_vad_pred.T, aspect='auto', origin='lower', 
                 extent=[0, len(sample_vad_pred)*0.02, 0, 4], 
                 cmap='Blues', alpha=0.7)
axes[1, 0].imshow(sample_vad_target.T, aspect='auto', origin='lower',
                 extent=[0, len(sample_vad_target)*0.02, 0, 4],
                 cmap='Reds', alpha=0.5)
axes[1, 0].set_title('Activité VAD: Prédiction (Bleu) vs Vérité (Rouge)')
axes[1, 0].set_xlabel('Temps (s)')
axes[1, 0].set_ylabel('Locuteur ID')
axes[1, 0].set_yticks(range(4))

# Subplot pour OSD
axes[1, 1].plot(time_frames, sample_osd_pred, label='Prédiction OSD', 
               color='blue', linewidth=2, alpha=0.8)
axes[1, 1].plot(time_frames, sample_osd_target, label='Vérité OSD', 
               color='red', linewidth=2, alpha=0.6)
axes[1, 1].set_title('Détection de Chevauchement (OSD)')
axes[1, 1].set_xlabel('Temps (s)')
axes[1, 1].set_ylabel('Probabilité de Chevauchement')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_ylim(0, 1)

plt.tight_layout()
training_plot_path = RESULTS_DIR / 'training_results.png'
plt.savefig(training_plot_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Graphique d'entraînement sauvé: {training_plot_path}")

In [ ]:
# === 3. Matrice de confusion pour classification ===
if hasattr(trainer.model, 'use_speaker_classifier') and trainer.model.use_speaker_classifier:
    print("\n📊 Analyse de la classification des locuteurs...")
    
    # Extraire les prédictions de classification
    all_speaker_preds = []
    all_speaker_targets = []
    
    trainer.model.eval()
    with torch.no_grad():
        for batch in val_loader:
            features = batch['features'].to(trainer.device)
            vad_labels = batch['vad_labels'].to(trainer.device)
            
            try:
                vad_pred, osd_pred, embeddings, speaker_logits = trainer.model(features, return_embeddings=True)
                
                # Créer des labels de locuteurs à partir des VAD labels
                speaker_targets = torch.argmax(vad_labels.sum(dim=1), dim=1)  # Locuteur le plus actif
                speaker_preds = torch.argmax(speaker_logits, dim=1)
                
                all_speaker_preds.extend(speaker_preds.cpu().numpy())
                all_speaker_targets.extend(speaker_targets.cpu().numpy())
                
            except Exception as e:
                print(f"   Erreur dans un batch: {e}")
                continue
    
    if all_speaker_preds and all_speaker_targets:
        from sklearn.metrics import confusion_matrix, classification_report
        
        # Matrice de confusion
        cm = confusion_matrix(all_speaker_targets, all_speaker_preds)
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=[f'Pred {i}' for i in range(4)],
                   yticklabels=[f'True {i}' for i in range(4)])
        plt.title('Matrice de Confusion - Classification des Locuteurs', fontsize=14, fontweight='bold')
        plt.xlabel('Prédiction')
        plt.ylabel('Vérité Terrain')
        
        confusion_path = RESULTS_DIR / 'speaker_confusion_matrix.png'
        plt.savefig(confusion_path, dpi=300, bbox_inches='tight')
        plt.show()
        
        # Rapport de classification
        print("\n📋 Rapport de Classification:")
        print(classification_report(all_speaker_targets, all_speaker_preds,
                                  target_names=[f'Locuteur {i}' for i in range(4)],
                                  digits=3))
        
        print(f"✅ Matrice de confusion sauvée: {confusion_path}")
    else:
        print("⚠️ Pas assez de données pour la matrice de confusion")
else:
    print("⚠️ Classificateur de locuteurs non activé")

In [ ]:
# === 4. Résumé final et comparaisons ===
print("\n" + "="*60)
print("📊 RÉSUMÉ FINAL DE L'ENTRAÎNEMENT")
print("="*60)

# Créer un résumé complet
final_summary = {
    'Configuration': {
        'Architecture': f"TCN {config['model']['hidden_channels']}",
        'Paramètres': f"{sum(p.numel() for p in trainer.model.parameters()):,}",
        'Batch Size': config['training']['batch_size'],
        'Epochs': len(train_losses),
        'Learning Rate': config['optimizer']['lr'],
        'Device': str(trainer.device)
    },
    'Résultats Finaux': {
        'Train Loss': f"{train_losses[-1]:.4f}" if train_losses else "N/A",
        'Val Loss': f"{val_losses[-1]:.4f}" if val_losses else "N/A",
        'Best Val Loss': f"{best_val_loss:.4f}",
        'DER': f"{metrics.get('der', 0):.2f}%",
        'F1 Score': f"{metrics.get('f1_score', 0):.3f}",
        'Frame Precision': f"{metrics.get('frame_precision', 0):.3f}",
        'Frame Recall': f"{metrics.get('frame_recall', 0):.3f}"
    },
    'Fichiers Générés': {
        'Meilleur Modèle': str(best_model_path) if best_model_path.exists() else "Non sauvé",
        'Métriques': str(metrics_file),
        'Graphiques': str(training_plot_path),
        'Configuration': str(config_file)
    }
}

# Affichage du résumé
for section, items in final_summary.items():
    print(f"\n🔹 {section}:")
    for key, value in items.items():
        print(f"   {key:.<25} {value}")

# Sauvegarde du résumé
summary_file = RESULTS_DIR / 'training_summary.json'
with open(summary_file, 'w') as f:
    json.dump(final_summary, f, indent=2)

print(f"\n💾 Résumé complet sauvé: {summary_file}")

# === 5. Recommandations d'amélioration ===
print("\n" + "="*60)
print("💡 RECOMMANDATIONS POUR AMÉLIORER LES PERFORMANCES")
print("="*60)

current_der = metrics.get('der', 100)
current_f1 = metrics.get('f1_score', 0)

recommendations = []

if current_der > 25:
    recommendations.append("🔧 DER élevé: Augmenter le nombre d'epochs ou réduire le learning rate")
if current_f1 < 0.7:
    recommendations.append("🔧 F1 faible: Essayer focal loss avec gamma plus élevé")
if len(train_losses) < 20:
    recommendations.append("⏰ Entraînement court: Augmenter le nombre d'epochs")
if config['training']['batch_size'] < 16:
    recommendations.append("📦 Batch size petit: Augmenter si possible pour améliorer la stabilité")

recommendations.extend([
    "📊 Utiliser plus de données AMI si disponibles",
    "🎯 Affiner les hyperparamètres avec Optuna",
    "🔄 Essayer l'ensemble de modèles",
    "📈 Implémenter la validation croisée",
    "🧠 Tester différentes architectures d'attention"
])

for i, rec in enumerate(recommendations[:7], 1):
    print(f"{i}. {rec}")

print("\n" + "="*60)
print("🎉 ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS!")
print("="*60)
print(f"📁 Tous les résultats sont sauvés dans: {RESULTS_DIR}")
print(f"🧠 Meilleur modèle disponible dans: {MODEL_DIR}")

if config.get('use_wandb', False):
    print(f"📊 Logs détaillés disponibles sur Weights & Biases")

## 🚀 Prochaines Étapes

### 📝 Pour Continuer l'Amélioration:

1. **📊 Données**: Télécharger le corpus AMI complet
2. **⚙️ Hyperparamètres**: Optimiser avec Optuna
3. **🎯 Architecture**: Tester différentes tailles de modèle
4. **📈 Ensembles**: Combiner plusieurs modèles
5. **🔄 Post-traitement**: Améliorer la segmentation finale

### 💾 Fichiers Générés:
- `models/checkpoints/best_model.pth` - Meilleur modèle
- `results/evaluation_metrics.json` - Métriques détaillées
- `results/training_results.png` - Graphiques d'entraînement
- `results/training_summary.json` - Résumé complet

### 🎯 Objectifs de Performance:
- **DER < 20%** sur AMI corpus (état de l'art: ~15-18%)
- **F1 > 0.8** pour la détection d'activité
- **Temps réel** pour l'inférence

---

**🎉 Félicitations! Vous avez entraîné avec succès un système de diarization moderne avec toutes les optimisations avancées!**